# LlamaIndex

In [ ]:
!pip install llama-index-llms-huggingface-api

## Load Token HuggingFace

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

token = os.getenv("HF_TOKEN")



In [2]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

llm = HuggingFaceInferenceAPI(
    model_name="Qwen/Qwen2.5-Coder-32B-Instruct",
    temperature=0.7,
    max_tokens=100,
    token=token,
)

llm.complete("Hello, how are you?")
# I am good, how can I help you today?

CompletionResponse(text=" I'm just a computer program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={}, raw=None, logprobs=None, delta=None)

# RAG with LlamaIndex

In [ ]:
!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

In [ ]:
#from huggingface_hub import login
#login()

## Setting up the persona database

In [3]:
from datasets import load_dataset
from pathlib import Path

dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])



## Loading and embedding persona documents

In [4]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

In [5]:
from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ]
)

# run the pipeline sync or async
nodes = await pipeline.arun(documents=documents[:10])
nodes

[TextNode(id_='9233d834-78bb-4ea1-b3ca-adf5b8f05a63', embedding=[-0.02226509526371956, -0.007676179055124521, 0.0419173389673233, -0.0007882601930759847, -0.011876478791236877, -0.025896089151501656, -0.001756496261805296, 0.04435310885310173, -0.06505507230758667, -0.05754375085234642, -0.020809248089790344, -0.040906354784965515, -0.06627724319696426, 0.025205498561263084, -0.0020893211476504803, -0.02068157307803631, 0.005952480714768171, 0.1012638583779335, -0.01521955244243145, 0.012223580852150917, 0.0069720130413770676, -0.047711119055747986, 0.033515796065330505, -0.03336881473660469, -0.01902220956981182, 0.015590609051287174, 0.016110096126794815, -0.01962619088590145, -0.02224571630358696, -0.11299940943717957, -0.060164887458086014, 0.010574395768344402, -0.007163442671298981, 0.0172534491866827, 0.059296537190675735, -0.003607271471992135, 0.0017253418918699026, 0.06262476742267609, 0.0035850501153618097, 0.006949390284717083, 0.03680037707090378, 0.021572081372141838, -0.

## Storing and indexing documents

In [6]:


import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)



10

In [7]:
# We can create a VectorStoreIndex from the vector store and use it to query the documents by passing the vector store and embedding model to the from_vector_store() method.

from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding

embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)

## Querying the index

In [8]:

from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
import nest_asyncio

nest_asyncio.apply()  # This is needed to run the query engine
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query(
    "Respond using a persona that describes author and travel experiences?"
)
response



Response(response='The author is an anthropologist or cultural expert deeply immersed in the study of Cypriot culture, history, and society. With extensive research and firsthand experience living in Cyprus, this individual has cultivated a profound understanding of the local people, their customs, and the unique way of life that defines the island.', source_nodes=[NodeWithScore(node=TextNode(id_='74dc4a68-c504-48f8-93af-6ed88082dda3', embedding=None, metadata={'file_path': '/home/arthur/Workspace/Perso/RL_Course/data/persona_1.txt', 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2025-03-10', 'last_modified_date': '2025-03-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNod

## Evaluation and observability

In [9]:
from llama_index.core.evaluation import FaithfulnessEvaluator

# query index
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing



True

In [12]:
import llama_index
import os
from dotenv import load_dotenv

load_dotenv()

PHOENIX_API_KEY = os.getenv("PHOENIX_API_KEY")
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix",
    endpoint="https://llamatrace.com/v1/traces"
)

Attempting to instrument while already instrumented


In [13]:


response = query_engine.query(
    "What is the name of the someone that is interested in AI and techhnology?"
)
response



Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


Response(response='The provided information does not mention anyone interested in AI and technology. Instead, it describes an anthropologist or cultural expert with a focus on Cypriot culture, history, and society.', source_nodes=[NodeWithScore(node=TextNode(id_='74dc4a68-c504-48f8-93af-6ed88082dda3', embedding=None, metadata={'file_path': '/home/arthur/Workspace/Perso/RL_Course/data/persona_1.txt', 'file_name': 'persona_1.txt', 'file_type': 'text/plain', 'file_size': 266, 'creation_date': '2025-03-10', 'last_modified_date': '2025-03-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='620a0dc3-0cc5-4622-b3d1-5d40362120a8', node_type='4', metadata={'file_path': '/home/arthur/Workspace/Perso/RL_Course/data/

# Agents Part 2

In [14]:
!pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

In [15]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream


def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b


llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

In [18]:
# stateless
response = await agent.run("What is 2 times 2?")

# remembering state
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)

Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to e

In [19]:


import chromadb

from llama_index.core import VectorStoreIndex
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface_api import HuggingFaceInferenceAPIEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore

# Create a vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a query engine
embed_model = HuggingFaceInferenceAPIEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

# Create a RAG agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions. ",
)



In [20]:


handler = query_engine_agent.run(
    "Search the database for 'science fiction' and return some persona descriptions."
)
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp



Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: personas
Action Input: {"input": "science fiction"}

Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 



Called tool:  personas {'input': 'science fiction'} => science fiction


Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


...


Called tool:  personas {'input': 'science fiction character archetypes'} => While the context provided focuses on a web developer or student with an interest in HTML, CSS, and website building, the query asks about science fiction character archetypes. In science fiction, character archetypes can include the visionary, the rebel, the mentor, the explorer, and the survivor. These roles often play crucial parts in the narrative, driving the plot and engaging the audience with their unique traits and motivations.


Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: In science fiction, character archetypes can include:

- **The Visionary**: Often a forward-thinking individual who sees possibilities beyond the current reality. They are key in driving technological advancements and new ideas.
- **The Rebel**: A character who challenges the status quo and fights against oppressive systems or entities. They are often seen as the protagonist or a key ally.
- **The Mentor**: A wise and experienced character who guides and teaches the protagonist or other characters. They provide crucial advice and support.
- **The Explorer**: Adventurous and curious, this character is always seeking new worlds, technologies, or knowledge. They are often the first to venture into uncharted territories.
- **The Survivor**: Enduring and resilient, this character has faced numerous challenges and has managed to overcome them. They often carry the weight of past experiences and

Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='In science fiction, character archetypes can include:\n\n- **The Visionary**: Often a forward-thinking individual who sees possibilities beyond the current reality. They are key in driving technological advancements and new ideas.\n- **The Rebel**: A character who challenges the status quo and fights against oppressive systems or entities. They are often seen as the protagonist or a key ally.\n- **The Mentor**: A wise and experienced character who guides and teaches the protagonist or other characters. They provide crucial advice and support.\n- **The Explorer**: Adventurous and curious, this character is always seeking new worlds, technologies, or knowledge. They are often the first to venture into uncharted territories.\n- **The Survivor**: Enduring and resilient, this character has faced numerous challenges and has managed to overcome them. The

## Creating multi-agent systems

In [24]:


from llama_index.core.agent.workflow import (
    AgentWorkflow,
    ReActAgent,
)


# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract],
    llm=llm,
)

query_agent = ReActAgent(
    name="info_lookup",
    description="Looks up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm,
)

# Create and run the workflow
agent = AgentWorkflow(agents=[calculator_agent, query_agent], root_agent="calculator")

# Run the system
handler = agent.run(user_msg="Can you add 5 and 3?")



Failed to export batch code: 401, reason: 


Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 
Failed to export batch code: 401, reason: 


In [25]:

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp



Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 5, "b": 3}
Called tool:  add {'a': 5, 'b': 3} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 8

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='8')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 5, 'b': 3}, tool_id='6a4536df-2674-4946-b6d4-64d6e4cda434', tool_output=ToolOutput(content='8', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 5, 'b': 3}}, raw_output=8, is_error=False), return_direct=False)], raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='8', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1741620681, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=None, object='chat.completion.chunk'), current_agent_name='calculator')

## Asyncio

In [28]:
import asyncio


async def fetch_data(delay):
    print(f"Started fetching data with {delay}s delay")

    # Simulates I/O-bound work, such as network operation
    await asyncio.sleep(0.1)

    print("Finished fetching data")
    return f"Data after {delay}s"


async def main():
    print("Starting main")

    # Schedule two tasks concurrently
    task1 = asyncio.create_task(fetch_data(2))
    task2 = asyncio.create_task(fetch_data(3))

    # Wait until both tasks complete
    result1, result2 = await asyncio.gather(task1, task2)

    print(result1)
    print(result2)
    print("Main complete")


if __name__ == "__main__":
    asyncio.run(main())

Starting main
Started fetching data with 2s delay
Started fetching data with 3s delay
Finished fetching data
Finished fetching data
Data after 2s
Data after 3s
Main complete
